In [1]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR  #
# from torch.utils.data import TensorDataset, \
#     DataLoader  # Own stuff set of data quality checks, tensor shapes might be different dataloader loads the tensor,
import pytorch_lightning as pl  #
from pytorch_lightning.callbacks.early_stopping import \
    EarlyStopping  # early stop when you reach optimum loss, 3 times in a row gradient descent
from pytorch_lightning.callbacks import LearningRateMonitor  # delta (loss / accuracy)
from pytorch_lightning.loggers import MLFlowLogger  # Model tracking
import torch
import torch.nn as nn
from transformers import DistilBertModel
from data_preprocessing import injestDataset, downSample
from data_cleaning import feature_cleaner, cleaningPreprocess
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

import torchmetrics
import pytorch_lightning as pl
from transformers import BertTokenizer
# from transformers import DistilBertModel

In [2]:
data = injestDataset()

In [3]:
df = downSample(data,"Recommended IND",1000)

In [4]:
df

,trueIndex,Review Text,Rating,Recommended IND
0,12849,I've been looking for sometime for a spring/su...,5,1
1,4578,Great spring time dress that looks good with a...,5,1
2,6169,This material of this shirt is soft and appeal...,4,1
3,2942,This dress is so light and twirly and the fabr...,4,1
4,2769,I bought this shirt for my 16 year old daughte...,4,1
...,...,...,...,...
995,12936,"This is a beautiful dress. however, it looked ...",1,0
996,258,"Fits well through the shoulders and arms, but ...",3,0
997,19148,I love maeve dresses but there are several iss...,3,0
998,20477,"Okay, so this top has its redeeming qualities,...",3,0


In [5]:
df = cleaningPreprocess(df, "Review Text")

In [6]:
df = df.rename(columns={"Recommended IND": "targetOne", "Review Text": "feature", "Rating": "targetTwo"})

In [7]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [8]:
# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, df,transform=None):
        self.input_data = df
        self.max_length = 30
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenize_dataframe()
        # self.split_columns()
        # # Perform one-hot encoding on the features
        # self.column_transformer = None
        # self.new_features = None
        # self.label_binarizer = LabelEncoder()
        # self.new_targetOne =  None
        # self.new_targetTwo =  Non

        # self.transform = transform 
        
    def train_val_test_idx(self):
        idx = list(range(len(self.raw_targetOne)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.raw_targetOne,
                                random_state=500)
        
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.5,
                                random_state=500)
        self.train_idx = train_idx 
        self.test_idx = test_idx
        self.val_idx = val_idx,
    
    def split_columns(sef):
        self.features = self.input_data["feature"]
        self.targetOne = self.input_data["targetOne"]
        self.targetTwo = self.input_data["targetTwo"]

    def tokenize_dataframe(self):
        tokenized_texts = []

        for text in self.input_data['feature']:
            encoded_inputs = self.tokenizer(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            tokenized_texts.append(encoded_inputs)
        
        self.input_data['tokenizedFeature'] = tokenized_texts


        # return train_idx, test_idx,  val_idx 

        # if self.train:
        #     self.indices = train_idx
            
        # elif self.test:
        #     self.indices =  test_idx
            
        # elif self.val:
        #     self.indices =  val_idx
        # else :
        #     self.indices = idx
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        sample = self.features[index], self.targetOne[index], self.targetTwo[index] 
        # if self.transform:
        #     sample = self.transform(sample)
        return sample

    
class to_tensor:
    def __call__(self,sample):
        features, targets= sample
        return torch.tensor(features,dtype=torch.float32,device='mps:0'), torch.tensor(targets ,dtype=torch.int32,  device='mps:0')
    
    
def transformInput(trainDataset,this_dict):
    return torch.tensor(trainDataset.column_transformer.transform(this_dict),dtype=torch.float32,device='mps:0')
    
#The loader outputs a tensor float64 mac uses mps:0 and float32


In [9]:
trainDataset = MyDataset(df)

In [10]:
trainDataset.input_data

,trueIndex,feature,targetTwo,targetOne,tokenizedFeature
0,12849,ive looking sometime springsummer jacket wear ...,5,1,"[input_ids, token_type_ids, attention_mask]"
1,4578,great spring time dress looks good thin top ca...,5,1,"[input_ids, token_type_ids, attention_mask]"
2,6169,material shirt soft appealing great fall side ...,4,1,"[input_ids, token_type_ids, attention_mask]"
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1,"[input_ids, token_type_ids, attention_mask]"
4,2769,bought shirt 16 year old daughter looked adora...,4,1,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0,"[input_ids, token_type_ids, attention_mask]"
996,258,fits well shoulders arms zero waist looks like...,3,0,"[input_ids, token_type_ids, attention_mask]"
997,19148,love maeve dresses several issues one taste 1 ...,3,0,"[input_ids, token_type_ids, attention_mask]"
998,20477,okay top redeeming qualities promise gave 3 st...,3,0,"[input_ids, token_type_ids, attention_mask]"


In [11]:
trainDataset.input_data["tokenizedFeature"][1]

{'input_ids': tensor([[  101,  2307,  3500,  2051,  4377,  3504,  2204,  4857,  2327,  4003,
         10762,  4906,  2978,  6065, 21698,  5156,  2946,  2215, 11754,  5017,
         21776, 13950,  2100,  2052,  2946,  2293,  8313,  2507,  2750,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}